In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob
from tqdm.notebook import tqdm, trange

"""Change to the data folder"""
new_path = "./new_train/new_train"

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [2]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [3]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [4]:
model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.nn.Sequential(
    torch.nn.Conv2d(4, 8, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Conv2d(8, 24, 5),
    torch.nn.ReLU(),
    torch.nn.MaxPool2d(2, 2),
    #torch.nn.Linear(24 * 5 * 5, 512),
    #torch.nn.ReLU(),
    #torch.nn.Linear(512, 256),
    #torch.nn.ReLU(),
    #torch.nn.Linear(256, 120)
)
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
class LSTM(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = torch.nn.LSTM(76, hidden_size=98, batch_first=True)
        self.linear = torch.nn.Linear(19*98, 7200)
        self.linear2 = torch.nn.Linear(5880, 6500)
        self.relu = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(6500, 6500)
        self.linear4 = torch.nn.Linear(6500, 7200)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = torch.flatten(out, start_dim=1)
        #predict = self.linear(out)
        #predict = torch.reshape(predict, torch.Size([64, 60, 30, 4]))
        out = self.relu(self.linear2(out))
        out = self.relu(self.linear3(out))
        out = self.relu(self.linear4(out))
        return out

model = LSTM()
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
model = torch.load('./models/1epochAdam.pt')
model.train()
model.to(device)
if cuda_status:
    model = model.cuda()

### Visualize the batch of sequences

In [5]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 3

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss).item())

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


591.3389282226562
510.1405029296875
364.1410217285156
380.54803466796875
339.64373779296875
263.7250671386719
207.22512817382812
241.1396942138672
235.02902221679688
194.53053283691406
187.10562133789062
138.29075622558594
128.87527465820312
100.16696166992188
78.4795150756836
70.75957489013672
59.284732818603516
51.26582336425781
44.05048751831055
36.15816879272461
37.134830474853516
36.5481071472168
45.3492317199707
60.45429229736328
58.32893371582031
46.97327423095703
34.26049041748047
30.284862518310547
24.438486099243164
22.49806022644043
19.69540023803711
17.0848388671875
21.54589080810547
15.746970176696777
12.987625122070312
15.638594627380371
13.63847827911377
13.88553237915039
12.062273979187012
11.95132064819336
9.97717571258545
11.85484790802002
11.121954917907715
14.438239097595215
17.14938735961914
27.699996948242188
46.73644256591797
65.38426971435547
64.41777801513672
67.64765167236328
69.47669219970703
44.88310241699219
37.89505386352539
28.550357818603516
30.567119598

40.277278900146484
27.749359130859375
30.800582885742188
21.518003463745117
27.037233352661133
26.401626586914062
28.056642532348633
33.45091247558594
36.870933532714844
24.609128952026367
19.214614868164062
20.347108840942383
23.356718063354492
27.54403305053711
34.26687240600586
31.09455680847168
30.72806739807129
30.829275131225586
31.178190231323242
24.42462921142578
18.43550682067871
15.041125297546387
14.350192070007324
14.99195384979248
15.479307174682617
8.985339164733887
8.442371368408203
11.08674430847168
12.351001739501953
12.341403007507324
14.423948287963867
17.949186325073242
20.324678421020508
24.524898529052734
28.666540145874023
20.24176597595215
20.388288497924805
30.323518753051758
43.213924407958984
30.29634666442871
25.77224349975586
17.252595901489258
13.523077011108398
13.354144096374512
12.433534622192383
11.758606910705566
14.339805603027344
15.029251098632812
17.496644973754883
18.291648864746094
18.687841415405273
16.43010711669922
18.775609970092773
15.15016

10.408207893371582
9.089292526245117
8.538784980773926
8.158997535705566
10.65070915222168
11.2975435256958
9.674598693847656
9.7103910446167
9.632647514343262
8.344672203063965
6.838675498962402
7.163899898529053
8.982710838317871
8.531081199645996
8.89802360534668
9.780816078186035
12.039338111877441
14.532703399658203
21.356258392333984
27.156709671020508
29.43937110900879
39.369102478027344
37.61640930175781
54.718082427978516
61.9666633605957
65.371826171875
57.04351043701172
52.80165481567383
45.74650192260742
55.15934753417969
46.99909591674805
39.22880935668945
14.730813026428223
15.514636993408203
17.443748474121094
20.899688720703125
17.119129180908203
21.702529907226562
18.81441307067871
18.85500144958496
15.27920913696289
18.440088272094727
22.716720581054688
25.94973373413086
26.005311965942383
35.0128059387207
24.058696746826172
25.163787841796875
30.366600036621094
40.39186096191406
47.4726448059082
61.045658111572266
45.36009216308594
41.1576042175293
17.754594802856445

9.127861022949219
9.400681495666504
7.6171112060546875
8.897871971130371
7.973166465759277
7.981858730316162
7.037312030792236
6.969513893127441
7.376958847045898
9.49114990234375
11.639394760131836
13.139945983886719
10.910870552062988
12.08969497680664
16.36188316345215
21.23627281188965
22.418888092041016
25.258010864257812
26.998205184936523
33.44584274291992
31.313156127929688
32.77125930786133
32.354331970214844
41.68947982788086
35.78799057006836
35.20857620239258
46.77165222167969
40.329429626464844
31.817428588867188
28.44019889831543
24.049943923950195
28.726715087890625
31.239992141723633
46.687923431396484
44.247276306152344
53.857078552246094
32.42512512207031
21.8680477142334
16.763275146484375
12.694793701171875
12.213449478149414
9.969342231750488
8.2186861038208
10.038126945495605
10.6347017288208
9.463020324707031
10.931924819946289
10.635299682617188
10.733054161071777
9.016732215881348
9.246521949768066
9.539690017700195
11.039238929748535
14.154894828796387
18.0598

25.930177688598633
27.800445556640625
29.77199935913086
34.379783630371094
28.19690704345703
32.32735061645508
28.541473388671875
23.7292423248291
20.39584732055664
19.268510818481445
16.392345428466797
17.78312873840332
19.778854370117188
17.6463623046875
16.63947296142578
18.688770294189453
12.528841972351074
8.026556015014648
6.45755672454834
6.221462726593018
5.2821149826049805
5.149555683135986
5.32208251953125
5.4966535568237305
6.381940841674805
7.362220764160156
6.752787113189697
7.212629795074463
9.163078308105469
11.687586784362793
13.932896614074707
15.69753646850586
17.56745147705078
23.725494384765625
31.94611930847168
51.56427764892578
47.54702377319336
51.26913070678711
54.58168029785156
63.57542419433594
22.911273956298828
16.4333553314209
14.583452224731445
15.183259010314941
13.993948936462402
12.285335540771484
12.299912452697754
12.776561737060547
14.515886306762695
18.17057228088379
25.510801315307617
30.12038230895996
29.759187698364258
27.69938850402832
29.958984

...

8.787331581115723
10.37529182434082
10.060213088989258
9.116289138793945
8.180543899536133
7.607625484466553
6.597752094268799
6.665823936462402
7.825440883636475
10.229144096374512
12.156359672546387
13.392577171325684
12.11819076538086
15.158720016479492
25.389846801757812
35.80489730834961
35.81804656982422
26.8629150390625
25.28717803955078
20.203439712524414
17.04166603088379
13.096699714660645
13.07351303100586
14.960426330566406
14.501669883728027
11.370450019836426
15.605413436889648
11.855192184448242
12.210790634155273
9.81502628326416
10.081003189086914
11.516083717346191
13.993552207946777
20.362863540649414
24.850337982177734
25.619155883789062
21.27440643310547
10.854726791381836
9.84807014465332
6.67665433883667
6.739532947540283
5.292797565460205
5.177453994750977
6.257515907287598
6.1293134689331055
4.837601661682129
5.522220134735107
5.093448162078857
5.4171462059021
4.349102020263672
4.440375804901123
4.171017169952393
4.818692684173584
5.668050765991211
7.0319747924

17.104093551635742
16.55742073059082
20.245201110839844
21.569677352905273
20.882234573364258
22.44004249572754
21.758028030395508
13.916069030761719
12.699115753173828
10.807236671447754
10.259577751159668
10.507627487182617
12.419785499572754
9.844120025634766
9.5592041015625
9.8739013671875
6.5074286460876465
4.023645877838135
3.3700809478759766
3.3430025577545166
3.540045738220215
3.5321097373962402
3.1786997318267822
3.4845879077911377
3.5671591758728027
3.9594058990478516
3.622685670852661
3.8746628761291504
4.879148960113525
5.560553073883057
5.862729072570801
6.466144561767578
7.612485885620117
8.983522415161133
12.492259979248047
19.979291915893555
22.991737365722656
27.969301223754883
40.42268753051758
46.10238265991211
23.00987434387207
13.662919044494629
11.185213088989258
12.220603942871094
10.382627487182617
9.665374755859375
8.63598918914795
7.874218940734863
8.558116912841797
9.979403495788574
12.434175491333008
13.880834579467773
12.177008628845215
11.528597831726074
1

16.048372268676758
18.528011322021484
19.596250534057617
20.222570419311523
15.790986061096191
15.161026000976562
8.390791893005371
5.992912769317627
5.484631538391113
4.982629776000977
3.5937230587005615
4.1878342628479
3.7882978916168213
2.9314019680023193
3.236027956008911
3.002960443496704
2.9987120628356934
2.7418031692504883
2.8343842029571533
2.560626983642578
2.8732950687408447
2.6609184741973877
2.7591488361358643
2.794355630874634
2.978844404220581
3.235793352127075
3.697795867919922
4.538057327270508
5.277252197265625
7.010167121887207
9.4821195602417
10.627083778381348
12.385139465332031
16.43798065185547
18.089550018310547
19.124910354614258
23.868701934814453
30.832651138305664
30.723215103149414
28.140634536743164
17.94417381286621
18.32526206970215
17.453458786010742
20.861225128173828
21.387054443359375
19.30052947998047
17.044668197631836
19.31476593017578
23.55022430419922
27.299072265625
26.112911224365234
25.41141128540039
19.97442054748535
17.450284957885742
10.25

4.5115966796875
5.535925388336182
6.068522930145264
5.438405990600586
6.052835941314697
6.247211456298828
6.189121246337891
4.9854559898376465
3.981598138809204
3.86750864982605
3.5561423301696777
3.592986822128296
3.4808273315429688
4.101999282836914
5.334102630615234
5.229530334472656
6.312582015991211
6.768925666809082
8.229239463806152
7.1633992195129395
8.04489517211914
9.736791610717773
12.539167404174805
16.99000358581543
19.243675231933594
19.43582534790039
17.342262268066406
12.845117568969727
10.480371475219727
10.701438903808594
11.835412979125977
10.469596862792969
11.588345527648926
11.081424713134766
9.560017585754395
7.273994445800781
8.529391288757324
11.469315528869629
18.605932235717773
27.48828125
28.85066795349121
24.81787872314453
16.946956634521484
14.7811861038208
20.227760314941406
25.204113006591797
25.462112426757812
13.648008346557617
11.40013313293457
10.720355987548828
10.341290473937988
10.66771411895752
11.333823204040527
12.485039710998535
12.61864566802

4.457620620727539
3.3518447875976562
3.1144213676452637
3.0036158561706543
2.907374143600464
2.793710470199585
2.6858911514282227
2.7636377811431885
2.440716505050659
2.7072036266326904
2.3723785877227783
2.3679049015045166
2.351680040359497
2.4520182609558105
2.4475722312927246
2.59736704826355
2.925673723220825
3.735053777694702
3.3003058433532715
3.5014798641204834
4.707547187805176
3.7761287689208984
3.3302295207977295
3.0076522827148438
3.1286914348602295
2.8093109130859375
2.9436535835266113
3.1895384788513184
3.550863265991211
3.9929683208465576
4.777846336364746
5.12544584274292
4.849237442016602
5.16641092300415
4.516904830932617
4.376368045806885
4.138889789581299
3.847177505493164
3.7894604206085205
4.992290019989014
5.847424507141113
7.47797966003418
9.69159984588623
11.145092010498047
11.056868553161621
8.267319679260254
8.493212699890137
11.247624397277832
10.33993911743164
11.996743202209473
15.587486267089844
14.649280548095703
16.319162368774414
25.482967376708984
41.0

In [ ]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 10
        
# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
#learning_rate =0.01
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        #x = torch.flatten(inp, start_dim=2)
        x = inp.permute(0,2,1,3).float()
        x = torch.flatten(x, start_dim=2)
        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None
        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        #y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


In [ ]:
torch.save(model, './models/12InitialWorkspace.pt')

In [ ]:
model = torch.load('./models/6epochmodel.pt')
model.eval()
model.to(device)
if cuda_status:
    model = model.cuda()

In [ ]:
import pandas as pd

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                